In [7]:
import numpy as np
import cv2 as cv
import cv2
import matplotlib.pyplot as plt

img1 = cv.imread('tir/00002.jpg', cv.IMREAD_GRAYSCALE)  # referenceImage
img2 = cv.imread('rgb/00002.jpg', cv.IMREAD_GRAYSCALE)  # sensedImage

# Initiate SIFT detector
sift_detector = cv.SIFT_create()
# Find the keypoints and descriptors with SIFT
kp1, des1 = sift_detector.detectAndCompute(img1, None)
kp2, des2 = sift_detector.detectAndCompute(img2, None)

# BFMatcher with default params
bf = cv.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)

# Filter out poor matches
good_matches = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good_matches.append(m)

matches = good_matches

points1 = np.zeros((len(matches), 2), dtype=np.float32)
points2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches):
    points1[i, :] = kp1[match.queryIdx].pt
    points2[i, :] = kp2[match.trainIdx].pt

# Find homography
H, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

# Warp image 1 to align with image 2
img1Reg = cv2.warpPerspective(img1, H, (img2.shape[1], img2.shape[0]))
cv.imwrite('aligned_img1.jpg', img1Reg)
# The problem is that this matrix H is found via a compute-intensive optimization process.

True

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

img1 = cv.imread('image1.jpg', cv.IMREAD_GRAYSCALE)  # referenceImage
img2 = cv.imread('image2.jpg', cv.IMREAD_GRAYSCALE)  # sensedImage

#  Resize the image by a factor of 8 on each side. If your images are 
# very high-resolution, you can try to resize even more, but if they are 
# already small you should set this to something less agressive.
resize_factor = 1.0/8.0

img1_rs = cv.resize(img1, (0,0), fx=resize_factor, fy=resize_factor)
img2_rs = cv.resize(img2, (0,0), fx=resize_factor, fy=resize_factor)

# Initiate SIFT detector 
sift_detector = cv.SIFT_create()

# Find the keypoints and descriptors with SIFT on the lower resolution images
kp1, des1 = sift_detector.detectAndCompute(img1_rs, None)
kp2, des2 = sift_detector.detectAndCompute(img2_rs, None)

# BFMatcher with default params
bf = cv.BFMatcher()
matches = bf.knnMatch(des1, des2, k=2)

# Filter out poor matches
good_matches = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good_matches.append(m)

matches = good_matches
points1 = np.zeros((len(matches), 2), dtype=np.float32)
points2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches):
    points1[i, :] = kp1[match.queryIdx].pt
    points2[i, :] = kp2[match.trainIdx].pt

# Find homography
H, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

# Get low-res and high-res sizes
low_height, low_width = img1_rs.shape
height, width = img1.shape
low_size = np.float32([[0, 0], [0, low_height], [low_width, low_height], [low_width, 0]])
high_size = np.float32([[0, 0], [0, height], [width, height], [width, 0]])

# Compute scaling transformations
scale_up = cv.getPerspectiveTransform(low_size, high_size)
scale_down = cv.getPerspectiveTransform(high_size, low_size)

#  Combine the transformations. Remember that the order of the transformation 
# is reversed when doing matrix multiplication
# so this is actualy scale_down -> H -> scale_up
h_and_scale_up = np.matmul(scale_up, H)
scale_down_h_scale_up = np.matmul(h_and_scale_up, scale_down)

# Warp image 1 to align with image 2
img1Reg = cv2.warpPerspective(
            img1, 
            scale_down_h_scale_up, 
            (img2.shape[1], img2.shape[0])
          )

cv.imwrite('aligned_img1.jpg', img1Reg)

In [12]:
import cv2

def enhance_contrast(img):
    # 对图像进行对比度增强
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced_img = clahe.apply(img)
    return enhanced_img

def double_light_fusion(visible_img, infrared_img):
    # 可见光图像对比度增强
    visible_enhanced = enhance_contrast(visible_img)
    
    # 红外图像对比度增强
    infrared_enhanced = enhance_contrast(infrared_img)
    
    # 将对比度增强后的两个图像进行融合
    fused_img = cv2.addWeighted(visible_enhanced, 0.5, infrared_enhanced, 0.5, 0)
    
    return fused_img

# 读取可见光图像和红外图像
visible_img = cv2.imread('rgb/00030.jpg', cv2.IMREAD_GRAYSCALE)
infrared_img = cv2.imread('tir/00030.jpg', cv2.IMREAD_GRAYSCALE)

# 进行双光融合
fused_image = double_light_fusion(visible_img, infrared_img)

# 显示融合后的图像
cv2.imshow('Fused Image', fused_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv.imwrite('fused_image.jpg', fused_image)

True

In [25]:
import cv2
import numpy as np

# 读取两个输入图像
img_optical = cv2.imread('rgb/00001.jpg', 0)  # 光学图像
img_ir = cv2.imread('tir/00001.jpg', 0)            # 红外图像

# 使用SIFT检测特征点并计算描述符
sift = cv2.SIFT_create()
keypoints_optical, descriptors_optical = sift.detectAndCompute(img_optical, None)
keypoints_ir, descriptors_ir = sift.detectAndCompute(img_ir, None)

# 使用FLANN匹配器匹配特征点
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(descriptors_optical, descriptors_ir, k=2)

# 去除错误匹配
good_matches = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good_matches.append(m)

# 提取配准点坐标
src_pts = np.float32([keypoints_optical[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints_ir[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
print(dst_pts)#只有白天的才能找到配准点

# 计算变换矩阵
M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

# 对光学图像进行透视变换，使其与红外图像配准
registered_img_optical = cv2.warpPerspective(img_optical, M, (img_ir.shape[1], img_ir.shape[0]))

# 融合图像
blended_img = cv2.addWeighted(registered_img_optical, 0.5, img_ir, 0.5, 0)


# 保存显示结果
cv.imwrite('blended_img_01.jpg', blended_img)
cv2.imshow('Blended Image', blended_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


[]


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'cv::findHomography'


In [26]:
good_matches

[]

In [32]:
import cv2
import numpy as np

# 读取输入图像
img_ir = cv2.imread('tir/00001.jpg', 0)  # 红外图像
img_optical = cv2.imread('rgb/00001.jpg', 0)  # 光学图像

# 使用SIFT检测特征点并计算描述符（仅用于红外图像）
sift = cv2.SIFT_create()
keypoints_ir, descriptors_ir = sift.detectAndCompute(img_ir, None)

keypoints_optical, descriptors_optical = sift.detectAndCompute(img_optical, None)

# 使用FLANN匹配器匹配特征点
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)

# 如果光学图像的特征点数太少，则使用红外图像的特征点进行匹配
if len(keypoints_optical) >= 10:
    # keypoints_optical, descriptors_optical = sift.detectAndCompute(img_optical, None)
    matches = flann.knnMatch(descriptors_optical, descriptors_ir, k=2)
else:
    matches = flann.knnMatch(descriptors_ir, descriptors_ir, k=2)

# 去除错误匹配
good_matches = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good_matches.append(m)

# 提取配准点坐标
if len(keypoints_optical) >= 10:
    src_pts = np.float32([keypoints_optical[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
else:
    src_pts = np.float32([keypoints_ir[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints_ir[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# 计算变换矩阵
M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

# 对光学图像进行透视变换，使其与红外图像配准
registered_img_optical = cv2.warpPerspective(img_optical, M, (img_ir.shape[1], img_ir.shape[0]))

# 融合图像
blended_img = cv2.addWeighted(registered_img_optical, 0.5, img_ir, 0.5, 0)

# 显示结果
cv.imwrite('blended_img_01.jpg', blended_img)
cv2.imshow('Blended Image', blended_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\fundam.cpp:385: error: (-28:Unknown error code -28) The input arrays should have at least 4 corresponding point sets to calculate Homography in function 'cv::findHomography'


In [37]:
import cv2
import numpy as np

# 读取输入图像
img_ir = cv2.imread('tir/00847.jpg', 0)  # 红外图像
img_optical = cv2.imread('rgb/00847.jpg', 0)  # 光学图像

# 使用SIFT检测特征点并计算描述符（红外图像）
sift = cv2.SIFT_create()
keypoints_ir, descriptors_ir = sift.detectAndCompute(img_ir, None)

# 使用FLANN匹配器匹配特征点
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)

# 使用红外图像的特征点进行匹配
matches = flann.knnMatch(descriptors_ir, descriptors_ir, k=2)

# 去除错误匹配
good_matches = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good_matches.append(m)

# 提取配准点坐标
src_pts = np.float32([keypoints_ir[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints_ir[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# 计算变换矩阵
M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

# 对光学图像进行透视变换，使其与红外图像配准
registered_img_optical = cv2.warpPerspective(img_optical, M, (img_ir.shape[1], img_ir.shape[0]))

# 融合图像
blended_img = cv2.addWeighted(registered_img_optical, 0.5, img_ir, 0.5, 0)

# 显示结果
cv.imwrite('blended_img_847.jpg', blended_img)
cv2.imshow('Blended Image', blended_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
